In [1]:
# changing core directory
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path:
    sys.path.append(dir1)
os.chdir('..')

In [2]:
import os
import logging
import warnings
import yaml

from replay_benchmarks.utils.conf import load_config, seed_everything
from replay_benchmarks import TrainRunner, InferRunner

import logging
import os
from abc import ABC, abstractmethod
from typing import Tuple

import pandas as pd
from rs_datasets import MovieLens, Netflix

from replay.data import (
    FeatureHint,
    FeatureInfo,
    FeatureSchema,
    FeatureSource,
    FeatureType,
    Dataset,
)
from replay.preprocessing.filters import MinCountFilter, NumInteractionsFilter
from replay.splitters import TimeSplitter
from replay.utils import DataFrameLike
from replay.data.nn import (
    SequenceTokenizer,
    SequentialDataset,
    TensorFeatureSource,
    TensorSchema,
    TensorFeatureInfo,
)

from torch.utils.data import DataLoader
from replay.models.nn.sequential.sasrec import (
    SasRecTrainingDataset,
    SasRecValidationDataset,
    SasRecPredictionDataset,
)
from replay.models.nn.sequential.bert4rec import (
    Bert4RecTrainingDataset,
    Bert4RecValidationDataset,
    Bert4RecPredictionDataset,
)



import logging
import os

import lightning as L
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader
from torch.profiler import profile, record_function, ProfilerActivity, schedule

from replay_benchmarks.base_runner import BaseRunner
from replay.metrics import OfflineMetrics, Recall, Precision, MAP, NDCG, HitRate, MRR
from replay.metrics.torch_metrics_builder import metrics_to_df
from replay.models.nn.sequential import SasRec, Bert4Rec
from replay.models.nn.optimizer_utils import FatOptimizerFactory
from replay.models.nn.sequential.callbacks import ValidationMetricsCallback
from replay.models.nn.sequential.postprocessors import RemoveSeenItems
from replay.models.nn.sequential.sasrec import (
    SasRecTrainingDataset,
    SasRecValidationDataset,
    SasRecPredictionDataset,
)
from replay.models.nn.sequential.bert4rec import (
    Bert4RecTrainingDataset,
    Bert4RecValidationDataset,
    Bert4RecPredictionDataset,
)

from replay_benchmarks.preprocessing import DatasetManager

In [3]:
config_dir = "./replay_benchmarks/configs"
base_config_path = os.path.join(config_dir, "config.yaml")
config = load_config(base_config_path, config_dir)

seed_everything(config["env"]["SEED"])

In [4]:
config = config
model_name = config["model"]["name"]
dataset_name = config["dataset"]["name"]
dataset_cfg = config["dataset"]
model_cfg = config["model"]["params"]
mode = config["mode"]["name"]
item_column = dataset_cfg["feature_schema"]["item_column"]
user_column = dataset_cfg["feature_schema"]["query_column"]
timestamp_column = dataset_cfg["feature_schema"]["timestamp_column"]
tokenizer = None
interactions = None
user_features = None
item_features = None


os.environ["CUDA_DEVICE_ORDER"] = config["env"]["CUDA_DEVICE_ORDER"]
os.environ["OMP_NUM_THREADS"] = config["env"]["OMP_NUM_THREADS"]
os.environ["CUDA_VISIBLE_DEVICES"] = config["env"]["CUDA_VISIBLE_DEVICES"]
os.environ["KAGGLE_USERNAME"] = "recsysaccelerate"
os.environ["KAGGLE_KEY"] = "6363e91b656fea576c39e4f55dcc1d00"

dataset_manager = DatasetManager(config)

In [5]:
# splits = dataset_manager.load_data()
# train_events, validation_events, validation_gt, test_events, test_gt = (
#     splits["train"],
#     splits["validation"],
#     splits["validation_gt"],
#     splits["test"],
#     splits["test_gt"],
# )

In [6]:
logging.info("Preprocessing data as splits are not cached.")
raw_data = dataset_manager._load_raw_data()
filtered_data = dataset_manager._filter_data(raw_data)

In [7]:
filtered_data.groupby(user_column).size().mean()

88.21638494184447